# Chapter 2: Functions

<div id="toc"></div>

## Item 14: Prefer Exceptions to Returning None

In [5]:
def divide(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return None

In [11]:
divide(1,2)

0.5

In [9]:
divide(1,0)

In [13]:
x, y = 1, 0
result = divide(x, y)
if result is None:
    print('Invalid inputs')

Invalid inputs


In [22]:
def divide(a, b):
    try:
        return a / b
    except:
        print('oops! something is wrong...')

In [23]:
divide(1,0)

oops! something is wrong...


In [ ]:
def divide(a, b):
    try:
        return True, a / b
    except ZeroDivisionError:
        return False, None

In [ ]:
success, result = divide( x, y)
if not success:
    print('Invalid inputs')

In [ ]:
_, result = divide(x, y)
if not result:
    print('Invalid inputs')

In [ ]:
def divide(a, b):
    try:
        return a / b
    except ZeroDivisionError as e:
        raise ValueError('Invalid inputs') from e

In [ ]:
x, y = 5, 2
try:
    result = divide(x, y)
except ValueError:
    print('Invalid inputs')
else:
    print('Result is %. 1f' % result)

* Functions that return None to indicate special meaning are error prone because None and other values (e.g., zero, the empty string) all evaluate to False in conditional expressions.  
* Raise exceptions to indicate special situations instead of returning None.  
* Expect the calling code to handle exceptions properly when they’re documented.  



## Item 15: Know How Closures Interact with Variable Scope

In [24]:
def sort_priority(values, group):
    def helper(x) :
        if x in group:
            return (0, x)
        return (1, x)
    values.sort(key=helper)

In [25]:
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}
sort_priority(numbers, group)
print(numbers)

[2, 3, 5, 7, 1, 4, 6, 8]


In [28]:
def sort_priority2(numbers, group):
    found = False
    def helper(x):
        if x in group:
            found = True # Seems simple
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found

In [27]:
found = sort_priority2(numbers, group)
print('Found: ', found)
print(numbers)

Found:  False
[2, 3, 5, 7, 1, 4, 6, 8]


In [31]:
def sort_priority2(numbers, group):
    found = False # Scope: 'sort_priority2'
    def helper(x) :
        if x in group:
            found = True # Scope: ‘ helper’ — Bad!
            return (0, x)
        return ( 1, x)
    numbers.sort( key=helper)
    return found

In [30]:
def sort_priority3(numbers, group):
    found = False
    def helper(x) :
        nonlocal found
        if x in group:
            found = True
            return (0, x)
        return (1, x)
    numbers.sort( key=helper)
    return found

In [32]:
class Sorter(object):
    def __init__(self, group):
        self.group = group
        self.found = False
    def __call__(self, x):
        if x in self. group:
            self.found = True
            return (0, x)
        return ( 1, x)
sorter = Sorter( group)
numbers.sort( key=sorter)
assert sorter.found is True

In [ ]:
# Python 2
def sort_priority(numbers, group):
    found = [False]
    def helper(x):
        if x in group:
            ound[0] = True
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found[0]

* Closure functions can refer to variables from any of the scopes in which they were defined.  
* By default, closures can’t affect enclosing scopes by assigning variables.  
* In Python 3, use the nonlocal statement to indicate when a closure can modify a variable in its enclosing scopes.  
* In Python 2, use a mutable value (like a single-item list) to work around the lack of the nonlocal statement.  
* Avoid using nonlocal statements for anything beyond simple functions.  


## Item 16: Consider Generators Instead of Returning Lists

In [52]:
def index_words(text):
    result = []
    if text:
        result.append(0)
    for index, letter in enumerate(text):
        if letter ==' ':
            result.append(index + 1)
    return result

In [53]:
address = 'Four score and seven years ago...'
result = index_words(address)
print( result[:3] )

[0, 5, 11]


In [54]:
def index_words_iter(text):
    if text:
        yield 0
    for index, letter in enumerate(text) :
        if letter == ' ' :
            yield index + 1

In [57]:
result = list(index_words_iter(address))
result

[0, 5, 11, 15, 21, 27]

In [58]:
def index_file( handle):
    offset = 0
    for line in handle:
        if line:
            yield offset
        for letter in line:
            offset += 1
            if letter == ' ':
                yield offset

In [ ]:
with open('./tmp/address. txt',' r') as f:
    it = index_file( f)
    results = islice(it, 0, 3)
    print(list(results) )

* Using generators can be clearer than the alternative of returning lists of accumulated results.  
* The iterator returned by a generator produces the set of values passed to yield expressions within the generator function’s body.  
* Generators can produce a sequence of outputs for arbitrarily large inputs because their working memory doesn’t include all inputs and outputs.  



## Item 17: Be Defensive When Iterating Over Arguments

In [60]:
def normalize( numbers) :
    total = sum( numbers)
    result = []
    for value in numbers:
        percnet = 100 * value / total
        result.append(percent)
    return result

In [ ]:
visits = [15, 35, 80]
percentages = normalize(visits)
print(percentages)

In [ ]:
def read_visits(data_path) :
    with open(data_path) as f:
        for line in f:
            yield int(line)

In [ ]:
it = read_visits('/tmp/my_numbers.txt')
percentages = normalize(it)
print(percentages)

In [ ]:
it = read_visits('/tmp/my_numbers.txt')
print(list(it))
print(list(it)) # Already exhausted

In [ ]:
def normalize_copy( numbers) :
    numbers = list( numbers) # Copy the iterator
    total = sum( numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append( percent)
    return result

In [ ]:
it = read_visits('/tmp/my_numbers.txt')
percentages = normalize_copy( it)
print( percentages)

In [ ]:
def normalize_func( get_iter):
    total = sum( get_iter() ) # New iterator
    result = []
    for value in get_iter() : # New iterator
        percent = 100 * value / total
        result.append( percent)
    return result

In [ ]:
percentages = normalize_func(lambda: read_visits(path))

In [ ]:
class ReadVisits(object):
    def __init__(self, data_path):
        self. data_path = data_path
    def __iter__(self):
        with open(self.data_path) as f:
            for line in f:
                yield int(line)

In [ ]:
visits = ReadVisits(path)
percentages = normalize(visits)
print(percentages)

In [ ]:
def normalize_defensive(numbers):
    if iter(numbers) is iter(numbers): # An iterator — bad!
        raise TypeError('Must supply a container' )
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append( percent)
    return result

In [ ]:
visits = [15, 35, 80]
normalize_defensive(visits) # No error
visits = ReadVisits(path)
normalize_defensive(visits) # No error

In [ ]:
it = iter(visits)
normalize_defensive(it)

* Beware of functions that iterate over input arguments multiple times.  
* If these arguments are iterators, you may see strange behavior and missing values.  
* Python’s iterator protocol defines how containers and iterators interact with the iter and next built-in functions, for loops, and related expressions.  
* You can easily define your own iterable container type by implementing the ```__iter__``` method as a generator.  
* You can detect that a value is an iterator (instead of a container) if calling iter on it twice produces the same result, which can then be progressed with the next built in function.  


## Item 18: Reduce Visual Noise with Variable Positional Arguments

In [63]:
def log(message, values):
    if not values:
        print(message)
    else:
        values_str = ','.join(str(x) for x in values)
        print('%s: %s' % (message, values_str))
log('My numbers are', [1, 2])
log('Hi there', [] )

My numbers are: 1,2
Hi there


In [64]:
def log( message, *values) : # The only difference
    if not values:
        print( message)
    else:
        values_str =','.join(str(x) for x in values)
        print('%s: %s' % ( message, values_str) )
log('My numbers are' , 1, 2)
log('Hi there') # Much better

My numbers are: 1,2
Hi there


In [65]:
favorites = [7, 33, 99]
log( 'Favorite colors' , *favorites)

Favorite colors: 7,33,99


In [62]:
def my_generator():
    for i in range(10):
        yield i
def my_func(*args) :
    print(args)
it = my_generator()
my_func(*it)

(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)


In [67]:
def log(sequence, message, *values) :
    if not values:
        print('%s: %s'%(sequence, message))
    else:
        values_str = ', '.join(str(x) for x in values)
        print('%s: %s: %s'%(sequence, message, values_str))
log(1, 'Favorites', 7, 33) # New usage is OK
log('Favorite numbers', 7, 33) # Old usage breaks

1: Favorites: 7, 33
Favorite numbers: 7: 33


* Functions can accept a variable number of positional arguments by using *args in the def statement.  
* You can use the items from a sequence as the positional arguments for a function with the * operator.  
* Using the * operator with a generator may cause your program to run out of memory and crash.  
* Adding new positional parameters to functions that accept *args can introduce hard-to-find bugs.  


## Item 19: Provide Optional Behavior with Keyword Arguments

In [ ]:
def remainder(number, divisor):
    return number % divisor

In [ ]:
assert remainder( 20, 7) == 6

In [ ]:
remainder( 20, 7)
remainder( 20, divisor=7)
remainder( number=20, divisor=7)
remainder( divisor=7, number=20)

In [ ]:
remainder( number=20, 7)

In [ ]:
remainder( 20, number=7)

In [ ]:
def flow_rate( weight_diff, time_diff):
    return weight_diff / time_diff

weight_diff = 0. 5
time_diff = 3
flow = flow_rate( weight_diff, time_diff)
print( '%. 3f kg per second' % flow)

In [ ]:
def flow_rate( weight_diff, time_diff, period) :
    return ( weight_diff / time_diff) * period

In [ ]:
flow_per_second = flow_rate( weight_diff, time_diff, 1)

In [ ]:
def flow_rate( weight_diff, time_diff, period=1) :
    return ( weight_diff / time_diff) * period

In [ ]:
flow_per_second = flow_rate( weight_diff, time_diff)
flow_per_hour = flow_rate( weight_diff, time_diff, period=3600)

In [ ]:
def flow_rate( weight_diff, time_diff, period=1, units_per_kg=1):
    return ( ( weight_diff / units_per_kg) / time_diff) * period

In [ ]:
pounds_per_hour = flow_rate( weight_diff, time_diff, period=3600, units_per_kg=2. 2)

In [ ]:
pounds_per_hour = flow_rate( weight_diff, time_diff, 3600, 2. 2)

* Function arguments can be specified by position or by keyword.  
* Keywords make it clear what the purpose of each argument is when it would be confusing with only positional arguments.  
* Keyword arguments with default values make it easy to add new behaviors to a function, especially when the function has existing callers.  
* Optional keyword arguments should always be passed by keyword instead of by position.  



## Item 20: Use None and Docstrings to Specify Dynamic Default Arguments

In [2]:
from datetime import datetime
from time import sleep
def log( message, when=datetime.now()):
    print('%s: %s'%(when, message) )
    
log('Hi there!')
sleep(1)
log('Hi again!')

2017-05-31 16:09:11.185501: Hi there!
2017-05-31 16:09:11.185501: Hi again!


In [4]:
def log( message, when=None):
    """Log a message with a timestamp.
    Args:
        message: Message to print.
        when: datetime of when the message occurred.
            Defaults to the present time.
    """
    when = datetime.now() if when is None else when
    print(' %s: %s'%(when, message) )

In [5]:
log('Hi there!')
sleep(1)
log('Hi again!')

 2017-05-31 16:10:43.391937: Hi there!
 2017-05-31 16:10:44.392001: Hi again!


In [6]:
def decode(data, default={}):
    try:
        return json.loads( data)
    except ValueError:
        return default

In [ ]:
foo = decode('bad data')
foo['stuff'] = 5
bar = decode('also bad')
bar['meep'] = 1
print('Foo:', foo)
print('Bar:', bar)

In [ ]:
def decode( data, default=None) :
    """Load JSON data from a string.
    Args:
        data: JSON data to decode.
        default: Value to return if decoding fails.
            Defaults to an empty dictionary.
    """    
    if default is None:
        default = {}
    try:
        return json.loads(data)
    except ValueError:
        return default

In [ ]:
foo = decode('bad data')
foo['stuff'] = 5
bar = decode('also bad')
bar['meep'] = 1
print('Foo:', foo)
print('Bar:', bar)

* Default arguments are only evaluated once: during function definition at module load time.  
* This can cause odd behaviors for dynamic values (like {} or [] ).  
* Use None as the default value for keyword arguments that have a dynamic value.  
* Document the actual default behavior in the function’s docstring.  



## Item 21: Enforce Clarity with Keyword-Only Arguments

In [ ]:
def safe_division( number, divisor, ignore_overflow, ignore_zero_division) :
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

In [ ]:
result = safe_division( 1, 10**500, True, False)
print( result)

In [ ]:
result = safe_division( 1, 0, False, True)
print( result)

In [ ]:
def safe_division_b( number, divisor, ignore_overflow=False, ignore_zero_division=False):

In [ ]:
safe_division_b( 1, 10**500, ignore_overflow=True)
safe_division_b( 1, 0, ignore_zero_division=True)

In [ ]:
safe_division_b( 1, 10**500, True, False)

In [ ]:
def safe_division_c( number, divisor, *, ignore_overflow=False, ignore_zero_division=False):

In [ ]:
safe_division_c( 1, 10**500, True, False)

In [ ]:
safe_division_c( 1, 0, ignore_zero_division=True) # OK
try:
    safe_division_c(1, 0)
except ZeroDivisionError:
    pass # Expected

In [ ]:
# Python 2
def print_args( *args, **kwargs):
    print 'Positional: ', args
    print 'Keyword: ', kwargs
print_args( 1, 2, foo='bar', stuff='meep')

In [ ]:
# Python 2
def safe_division_d( number, divisor, **kwargs):
    ignore_overflow = kwargs. pop('ignore_overflow', False)
    ignore_zero_div = kwargs. pop('ignore_zero_division', False)
    if kwargs:
        raise TypeError('Unexpected **kwargs: %r'% kwargs)
    # …

In [ ]:
safe_division_d( 1, 10)
safe_division_d( 1, 0, ignore_zero_division=True)
safe_division_d( 1, 10**500, ignore_overflow=True)

In [ ]:
safe_division_d( 1, 0, False, True)

In [ ]:
safe_division_d( 0, 0, unexpected=True)

* Keyword arguments make the intention of a function call more clear.  
* Use keyword-only arguments to force callers to supply keyword arguments for potentially confusing functions, especially those that accept multiple Boolean flags.  
* Python 3 supports explicit syntax for keyword-only arguments in functions.  
* Python 2 can emulate keyword-only arguments for functions by using **kwargs and manually raising TypeError exceptions.  
